# Building AI-powered search in Amazon DocumentDB Vector Search using Amazon Bedrock and DocumentDB Vector Search
_**Using Bedrock Titan embedding model and DocumentDB `Vector Search` for similarity search on Game recommendations**_

---

---

## Contents


1. [Background](#Background)
1. [Setup](#Setup)
1. [Bedrock Model Call Preparation](#Bedrock-model-call-prepration)
1. [DocumentDB](#DocumentDB-vector-text-search)
1. [Evaluate Search Results](#Evaluate-DocumentDB-vector-Search-text-Results)

## Background

In this notebook, we'll build the core components of a textually similar Products. Often people don't know what exactly they are looking for and in that case they just type an item description and hope it will retrieve similar items.

One of the core components of searching textually similar items is a fixed length sentence/word embedding i.e. a  “feature vector” that corresponds to that text. The reference word/sentence embedding typically are generated offline and must be stored so they can be efficiently searched. In this use case we are using Amazon Bedrock Titan(https://aws.amazon.com/cn/bedrock/titan/).

To enable efficient searches for textually similar items, we'll use Amazon Bedrock Titan to generate fixed length sentence embeddings i.e “feature vectors” and use the Nearest Neighbor search in Amazon DocumentDB (with MongoDB compatibility) using the Vector Search. DocumentDB Vector Search lets you store and search for points in vector space and find the "nearest neighbors" for those points. Use cases include recommendations (for example, an "other songs you might like" feature in a music application), image recognition, and fraud detection.


## Setup
Install required python libraries for the workshop


In [ ]:
!pip install -U pymongo  tqdm boto3 requests scikit-image

### Gaming demo data 
example: [
 {"url": "game picture url", 
 "name_en": "game name",
 "descriptions_en": ["Game Category:Free-to-Play;Hero Shooter;Multi Player;Shoote",
  "Game Introduction: xxxxxx", 
 "MATURE CONTENT DESCRIPTION: No violence or gore."], 
 "descriptions_en":["Game genres: Free to Play;Battle Royale;Multiplayer;Shoote",
  "Game Introduction: xxxxxx",
  "MATURE CONTENT DESCRIPTION: No violence or gore."], 
  "recommendation": "Mem: 8 GB RAM,DirectX Version: 11,Network: Broadband Internet Connectivity, Requirement: Need 35 GB available storage"}
]

In [ ]:
import urllib.request
import os
import json
import boto3
from multiprocessing import cpu_count
from tqdm.contrib.concurrent import process_map
filename = 'metadata_en.json'

with open(filename) as json_file:
    results = json.load(json_file)
if not os.path.exists(filename):
   print ("metadata_en.json file not exits")
results[0]
print(results[0])

## Bedrock Model Call Preparation
prepare for Bedrock Titan model call

In [ ]:
# for bedrock model call
%pip install langchain==0.0.305 --force-reinstall

In [4]:
#for bedrock model call
import json
import os
import sys

import boto3

bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

In [5]:
# for bedrock model call
from langchain.embeddings import BedrockEmbeddings

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_client)

In [41]:
# calculate your LLM(Claude3) execution time
import time

def timer_llm_claude3(prompt, if_print=1):
    start_time = time.time()
    body = json.dumps({
      "max_tokens": 4096,
      "messages": [{"role": "user", "content": prompt}],
      "anthropic_version": "bedrock-2023-05-31"
    })

    response = bedrock_client.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")

    response_body = json.loads(response.get("body").read())
    end_time = time.time()
    elapsed_time = end_time - start_time
    if if_print == 1:
        print("----------------------------------------- OutPut -----------------------------------------")
        print("Elapsed time: ", elapsed_time, "seconds")
    return response_body.get("content")[0]['text']

In [6]:
# for Bedrock Embedding model call

def generate_embeddings(data):
    r = bedrock_embeddings.embed_query(data)
    return r


## DocumentDB Vector and Text Search

vector search for Amazon DocumentDB (with MongoDB compatibility), a new built-in capability that lets you store, index, and search millions of vectors with millisecond response times within your document database.

One of the key benefits of using pgvector is that it allows you to perform similarity searches on large datasets quickly and efficiently. This is particularly useful in industries like e-commerce, where businesses need to be able to quickly search through large product catalogs to find the items that best match a customer's preferences. It supports exact and approximate nearest neighbor search, L2 distance, inner product, and cosine distance.

To further optimize your searches, you can also use DocumentDB Vector Search's indexing features. By creating indexes on your vector data, you can speed up your searches and reduce the amount of time it takes to find the nearest neighbors to a given vector.

As much, in this sesction, we would also create docDB Text Search index, we would do vector and text search simultaneously and combine docDB vector search and text search results as Two-way recall for Bedrock LLM input.

In [47]:
# Set up a connection to your Amazon DocumentDB (MongoDB compatibility) cluster and creating the database
import pymongo

client = pymongo.MongoClient(
"docdb-vector-search-*******.com:27017",
username="masteruser",
password="******",
retryWrites=False,
tls='true',
tlsCAFile='global-bundle.pem')
db = client.similarity
collection = db.games

In [ ]:
import pymongo
import boto3 
import json 


for x in results:
    description1 = ' '.join(x.get('descriptions_en', []))
    vector = generate_embeddings(description1)
    record = { "name": x.get('name'),
               "name_en": x.get('name_en'),
              "descriptions": description1,
              "descriptions_en": ' '.join(x.get('descriptions_en', [])),
              "recommendation":x.get('recommendation'),
          "url": x.get('url'),
          "descriptions_embeddings": vector}
    # print("record",record)
    rec_id1 = collection.insert_one(record)  

collection.create_index ([("descriptions_embeddings","vector")], vectorOptions={
"lists": 1,
"similarity": "euclidean",
"dimensions": 1536}) 

#print ("Vector embeddings has been successfully loaded into DocumentDB") 

In [ ]:
#DocumentDB Text Search
collection.create_index({"descriptions_en": "text"})

## Evaluate DocumentDB vector Search Results

In this step we will use Bedrock Titan to generate embeddings for the query and use the embeddings to search the DocumentDB to retrive the nearest neighbours and retrive the relevent product images.


In [ ]:
from skimage import io 
import matplotlib.pyplot as plt
import requests
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["instructions"], 
    template="""
Human:
You are an excellent game recommender and we need you to provide summaries and recommendations for games

<Objective>
- Please include the game titles and game categories in your summaries and recommendations
- Please provide game descriptions and perform optimized summarization for them
- Please specify the hardware configurations required to run the games
- Evaluate the suitability of the games for underage players
</Objective>

<instructions>
{instructions}
</instructions>
Your objective is to determine based on the goals specified in<Objective>, List the game titles, game categories, game descriptions, and user's system configurations specified in <instructions>, and determine whether each game is suitable for minors, no additional irrelevant text content is required

Assistant:"""
)


def similarity_search(search_text):
    en_response= timer_llm_claude3("You are a translator, translate the following into English, without any additional rhetoric, just translate. In the translated content, remove the word game："+search_text)
    print(en_response)   
    data = {"inputs": search_text}
    res1 = generate_embeddings(data['inputs'])
    #Vector Search （
    query = {"vectorSearch" : {"vector" : res1, "path": "descriptions_embeddings", "similarity": "euclidean", "k": 2}}
    projection = {
    "_id":0,
    "name":1,
    "name_en":1,
    "recommendation":1,
    "url":1,
    "descriptions":1,
    "descriptions_en":1,
    "descriptions_embeddings": 1}
    r = collection.aggregate([{'$search': query},{"$project": projection}])
    
    #Text Search （
    tsr = collection.find({"$text": {"$search": en_response}}, {"score": {"$meta": "textScore"}}).sort({"score": {"$meta": "textScore"}})

    plt.rcParams["figure.figsize"] = [7.50, 3.50]
    plt.rcParams["figure.autolayout"] = True
    
    # merge two-way recall result 
    merged_list = []
    for doc in tsr:
        merged_list.append(doc)
        
    for doc in r:
        merged_list.append(doc)

    unique_set = set() 
    result = []
    for item in merged_list:
        dict_str = str(item['name_en'])
        if dict_str not in unique_set and len(result)<2:
            unique_set.add(dict_str)
            result.append(item)
            
    for x in result:
        # Pass in values to the input variables
        prompt = multi_var_prompt.format(instructions="game name:"+x["name_en"] + ".\nGame Description:" +x["descriptions_en"] +".\nHardware configuration："+ x["recommendation"])
        response= timer_llm_claude3(prompt)
        print(response)
        url = x["url"].split('?')[0]
        urldata = requests.get(url).content
        a = io.imread(url)
        plt.imshow(a)
        plt.axis('off')
        plt.show()

   

Using the above function `similarity_search` , lets do some search

In [ ]:
similarity_search("Multiplayer Shooting Game")